# Tutorial for extracting graph represenatations of all images

## Step 1: Register images from the same FOV

In [ ]:
from numpy.random import shuffle
from pathlib import Path
from vessel_morphology.registration import ImageRegistration
import pickle
from os import mkdir
from os.path import exists
from vessel_morphology.utilities import get_mov_files

In [ ]:
#load dictionary for matched volumes
with open('test_data/matched_stacks_init.pickle', 'rb') as f:
    dic = pickle.load(f)
sorted(dic['67/XYZres387'])

In [ ]:
mouse_ids_path = Path('test_data/THY1')#each mouse has its own folder with raw data in it
mouse_ids = list(mouse_ids_path.glob('*?[0-9]/*res*?[0-9].tif'))#grab folder names/mouse ids
images = sorted([x.as_posix() for x in mouse_ids if '_0001' not in x.as_posix()])
images = [x for x in images if  any(y in x for y in list(dic.keys()))]

unused_keys = [x for x in list(dic.keys()) if not  any(x in y for y in images)]
print(len(images))
shuffle(images)
params = { 
    'out_directory': 'test_data/raw_warped/', # path to output directory
    'initial_filename_extension': '.tif', # initial filename extension of unregistered images
    'final_filename_extension': '_warped.tif', # modified filename extension following registration
    'timepoint_suffixes': ['_0001'], # list of all timepoint suffixes present ie _0001, _0002, _0003, ...
    'sigma': 2, # Smoothing sigma applied to iamges prior to registration, used by ANTs rigid registration
    'flip': True, # flip images based on direction of aquision to match the reference. ONly works for files that were origionally olympus oir files
    'dic': dic # dictionary of matched aqusitions
}

if not exists(params['out_directory']):
    mkdir(params['out_directory'])


In [ ]:
# Create an instance of the ImageRegistration class
registration = ImageRegistration(images = images, 
                                 out_directory = params['out_directory'], 
                                 initial_filename_extension = params['initial_filename_extension'], 
                                 final_filename_extension = params['final_filename_extension'], 
                                 timepoint_suffixes = params['timepoint_suffixes'], 
                                 sigma = params['sigma'], 
                                 flip = params['flip'], 
                                 dic = params['dic'])

In [ ]:
# Call the register_images method
registration.register_images()

## Step 2: Predict segmentation results

### !!! Faster With GPUs !!!

In [ ]:
from os.path import exists
from os import mkdir
from pathlib import Path
from re import sub
import warnings
from numpy.random import shuffle
from vessel_morphology.predict import PredictWarped

In [ ]:
# Define the path to the mouse data
image_path = Path('test_data/raw_warped')
images = list(image_path.glob('*res*.tif'))
images = sorted([x.as_posix() for x in images])
shuffle(images)

# Create data dictionaries for prediction
params = {
    "input_dir": "test_data/raw_warped", # imput directory of files to segment
    "output_dir": "test_data/raw_warped_seg", # output directory for files after segmentation
    "num_evals": 3, # number of evaluations for the ensemble to predict with
    "base_file_extension": ".tif", # base file extension
    "pred_file_extension": "_pred.npy", # predicted probabilities filename extension
    "mean_file_extension": "_mean.npy", # mean probabilities filename extension
}

if not exists(params['output_dir']):
    mkdir(params['output_dir'])

##################################################################################################################
# STD will be calcualted later on an allocation without GPUs, this takes up alot of CPU time leaving GPUs idol
##################################################################################################################

data_dict = [
    {"image": image_name}
    for image_name in images if not exists(sub(params["base_file_extension"], params["pred_file_extension"], sub(params["input_dir"], params["output_dir"], image_name)))
]

print(len(data_dict))

In [ ]:
PredictMatt = PredictWarped(data_dict, 
                            params,
                            parameter_file='hyperparameter_pickle_files/parameters436.pickle', 
                            spatial_dims=3, 
                            in_channels=2, 
                            out_channels=3, 
                            img_size=(128,128,128), 
                            feature_size=16, 
                            hidden_size=768, 
                            mlp_dim=3072, 
                            pos_embed="perceptron",
                            res_block=True, 
                            norm_name="instance",
                            spacing=(1, 1, 0.375), 
                            i_min=0, 
                            i_max=1024, 
                            b_min=0.0, 
                            b_max=1.0, 
                            clip=True, 
                            channel_dim=0)
PredictMatt.prediction()

## Step 3: Calculate STD of prediction

In [ ]:
from pathlib import Path
from re import sub
from os.path import exists
from os import mkdir
from numpy.random import shuffle
from vessel_morphology.vessel_graph import SaveStdFile

In [ ]:
params = {
    "data_directory": "test_data/raw_warped_seg", #location of output probabilities from prediction
    "in_directory": "test_data/raw_warped_seg",
    "out_directory": "test_data/raw_warped_seg",
    "init_tag": 'pred', # tag for the predicited probabilities array in step 2 "pred_file_extension"
    "final_tag": 'std' # tag to save the standard deviation of the predicted probabiliteis under
}

if not exists(params['out_directory']):
    mkdir(params['out_directory'])

In [ ]:
# Define the files list
path = Path(params["data_directory"])
files = list(path.glob('*_' + params['init_tag'] + '.npy'))
files = sorted([x.as_posix() for x in files])
files = [x for x in files if not exists(sub(params['in_directory'], params['out_directory'], sub(params['init_tag'], params['final_tag'], x)))]
print(len(files))

# Shuffle the files list
shuffle(files)

In [ ]:
 # Create an instance of the SaveStdFile class with the files list and other parameters
save_std = SaveStdFile(files, params['in_directory'], params['out_directory'], params['init_tag'], params['final_tag'])
# Save the standard deviation files
save_std.std()

## Step 4: Binarize Vascular predictions

In [ ]:
from pathlib import Path
from numpy.random import shuffle
from pathlib import Path
from os.path import exists
from os import mkdir
from vessel_morphology.vessel_graph import Binarize

In [ ]:
directory = Path('test_data/raw_warped_seg')
files  = directory.glob('*-*_mean.npy')
files = sorted([x.as_posix() for x in files])
print(len(files))

parameters = {
    'min_prob': 0.75, # minimum probability for binarization to classify a voxel as a vessel
    'max_var': 0.2, # maximum standard deviation for binarization to classify a voxel as a vessel
    'in_directory': "test_data/raw_warped_seg", # input directory
    'out_directory': "test_data/raw_warped_seg", # output directory
    'mean_tag': 'mean', # tag for mean probaility
    'std_tag': 'std', # tag for mean standard deviation of probability
    'seg_tag': 'seg' # output tag for segmentation
}

if not exists(params['out_directory']):
    mkdir(params['out_directory'])

shuffle(files)

In [ ]:
binarizer = Binarize(files = files,
                     min_prob = parameters['min_prob'],
                     max_var = parameters['max_var'], 
                     in_directory = parameters['in_directory'],
                     out_directory = parameters['out_directory'], 
                     mean_tag = parameters['mean_tag'],
                     std_tag = parameters['std_tag'],
                     seg_tag = parameters['seg_tag'])

binarizer.binarize_files()


## Step 4a: binarize neuron segmentation and calculate distances from every nonneuron pixel to the closest neuron

In [ ]:
import numpy as np
from pathlib import Path
from vessel_morphology.vessel_graph import NeuronDistanceCalculator
from os.path import exists
from os import mkdir
from re import sub

In [ ]:
directory = Path('test_data/raw_warped_seg')
files  = directory.glob('*-*_mean.npy')
files = sorted([x.as_posix() for x in files])
files = [x for x in files if not exists(sub('_mean.npy', '_seg_nrn.npy', x))]
print(len(files))

parameters = {
    'min_prob': 0.75,
    'max_var': 0.1,
    'in_directory': 'test_data/raw_warped_seg',
    'out_directory': 'test_data/raw_warped_seg',
    'mean_tag': 'mean',
    'std_tag': 'std',
    'neuron_distance_tag': 'seg_nrn_dst',
    'seg_tag': 'seg',
    'seg_nrn_tag': 'seg_nrn'   
}

if not exists(params['out_directory']):
    mkdir(params['out_directory'])

np.random.shuffle(files)

In [ ]:
n = NeuronDistanceCalculator(files, 
                             parameters['min_prob'], 
                             parameters['max_var'], 
                             parameters['in_directory'], 
                             parameters['out_directory'], 
                             parameters['mean_tag'], 
                             parameters['std_tag'], 
                             parameters['neuron_distance_tag'], 
                             parameters['seg_tag'], 
                             parameters['seg_nrn_tag'])
n.calculate_distance()

## Step 5: Calculate union of matched files

In [ ]:
from numpy.random import shuffle
from pathlib import Path
from re import sub
from os.path import exists
from os import mkdir
import pickle
from vessel_morphology.vessel_graph import SegmentationMaskUnion

In [ ]:
with open('test_data/matched_stacks.pickle', 'rb') as handle:
    dic = pickle.load(handle)
    
directory_seg = Path('test_data/raw_warped_seg')
images = list(directory_seg.glob('*_0001_warped_seg.npy'))
images = sorted([x.as_posix() for x in images])
images = [x for x in images if exists(sub('_0001','',x))]
images = [x for x in images if any(y in x for y in list(dic.keys()))]

params = {'IOU_thresh':0.2,
        'input_directory':'test_data/raw_warped_seg',
        'output_directory':'test_data/raw_warped_seg',
        'initial_suffix':'_warped_seg.npy',
        'final_suffix_mat':'_seg_warped_single.mat',
        'final_suffix_npy':'_seg_warped_single.npy',
        'timepoint_suffixes':['_0001'],
        'thresh_remove_small_comps':250,
        'thresh_fill_holes':1000,
        'n_interations_binary_closing':3,
        }

if not exists(params['output_directory']):
    mkdir(params['output_directory'])

shuffle(images)
print(len(images))

In [ ]:
# Create an instance of ImageProcessor and process the images
processor = SegmentationMaskUnion(images, 
                                         dic,
                                         params['input_directory'], 
                                         params['output_directory'], 
                                         params['initial_suffix'], 
                                         params['final_suffix_mat'], 
                                         params['final_suffix_npy'],
                                         params['timepoint_suffixes'], 
                                         params['IOU_thresh'], 
                                         params['thresh_fill_holes'], 
                                         params['thresh_remove_small_comps'],
                                         params['n_interations_binary_closing'],
                                        )
processor.process_images()

## Step 6: Generate Skeletons from the Union of the segmentation masks

Run the following code in matlab after modifying the following matlab file gen_skeletons_warped_single.m

module load matlab 


matlab -nodisplay -nodesktop -r "gen_skeletons_warped_single"

## Step 7: Convert skeletons into graphs

In [4]:
from numpy.random import shuffle
from pathlib import Path
import pickle
#from vessel_morphology.vessel_graph import GraphGenerator

In [5]:
with open('test_data/matched_stacks.pickle', 'rb') as handle:
        dic = pickle.load(handle)
        
directory = Path('test_data/raw_warped_seg')
files_seg_0001 = directory.glob('*_0001_warped_seg.npy')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
files_seg_0001 = [x for x in files_seg_0001 if  any(y in x for y in list(dic.keys()))]
shuffle(files_seg_0001)
print(len(files_seg_0001))

params = {'IOU_thresh':0.2,
        'initial_suffix':'_warped_seg.npy',
        'initial_suffix_mat':'_skel_warped_single.mat',
        'final_suffix_npy':'_seg_warped_single.npy',
        'final_suffix_pickle':'_warped.pickle',
        'final_suffix_tif':'_single_skel.tif',
        'timepoint_suffixes':['_0001'],
        'ref_timepoint':'_0001',
        'thresh_remove_terminal_segemnts':20,
        'input_directory':'raw_warped_single_upsampled_seg',
        'output_directory':'raw_warped_single_upsampled_seg',
        }

1


In [6]:
files_seg_0001

['test_data/raw_warped_seg/67-XYZres387_0001_warped_seg.npy']

In [11]:
import os
from re import sub
from scipy.io import savemat, loadmat
import numpy as np
from sknw import build_sknw
from importlib.metadata import version as ver
#skel_file = 'test_data/raw_warped_seg/67-XYZres387_0001_skel_warped_single.mat'
skel_file = sub(params['ref_timepoint'] + params['initial_suffix'], params['initial_suffix_mat'], files_seg_0001[0]) 
os.path.exists(skel_file)
skel = loadmat(skel_file)['FilteredImage']

import threading
#print(ver(threading))

import faulthandler
faulthandler.enable()
PYTHONFAULTHANDLER = 1


#if np.sum(skel) != 0:
#    # Build a graph representation of the skeleton
#    graph = build_sknw(skel)

In [8]:
import numba
print(numba.__version__)
import networkx
print(networkx.__version__)
import numpy
print(numpy.__version__)
import sknw
print(sknw.__version__)
#import importlib_metadata
#print(importlib_metadata.__version__)
#import zipp
#print(zipp.__version__)
import llvmlite
print(llvmlite.__version__)
import setuptools
print(setuptools.__version__)

0.56.3
2.6.3
1.23.0
0.1
0.39.1
69.5.1


In [2]:
import package_deps
package = package_deps.find('sknw')
print(package.dependencies)

ModuleNotFoundError: No module named 'package_deps'

In [ ]:
import faulthandler
import sys
sys.settrace

MakeGraphs = GraphGenerator(files = files_seg_0001,
                            dic = dic, 
                            in_directory=params['input_directory'], 
                            out_directory = params['output_directory'], 
                            in_suffix = params['initial_suffix'], 
                            final_suffix_pickle = params['final_suffix_pickle'], 
                            in_suffix_mat = params['initial_suffix_mat'], 
                            final_suffix_tif = params['final_suffix_tif'], 
                            ref_timepoint = params['ref_timepoint'], 
                            timepoint_suffixes = params['timepoint_suffixes'], 
                            IOU_thresh = params['IOU_thresh'], 
                            thresh_remove_terminal_segemnts = params['thresh_remove_terminal_segemnts'])
faulthandler.enable()
MakeGraphs.generate_graphs()

## Step 8: Calculate radii for vessel segments

In [ ]:
from numpy import array
from numpy.random import shuffle
from pathlib import Path
from re import sub
from os.path import exists
from os import mkdir
from vessel_morphology.vessel_graph import VesselRadiiCalc

In [ ]:
params = {
    'in_directory': 'test_data/raw_warped_seg',
    'img_directory': 'test_data/raw_warped',
    'mean_directory': 'test_data/raw_warped_seg',
    'std_directory': 'test_data/raw_warped_seg',
    'out_directory': 'test_data/graphs',
    'pickle_file_suffix': '_warped.pickle',
    'out_pickle_suffix': '_radii.pickle',
    'img_suffix': '_warped.tif',
    'seg_suffix': '_warped_seg.npy',
    'mean_suffix': '_warped_mean.npy',
    'std_suffix': '_warped_std.npy',
    'neuron_distance_suffix': '_warped_seg_nrn_dst.npy',
    'second_channel': True,
    'neuron_channel': True,
    'psf': array([0.636,0.127,0.127]),
    'spacing': (1,1,2.645833333),
    'vessel_segment_limit': 2000,
    'max_pixel_value': 1023,
    'n_iter_deconv': 10,
    'grid_size_psf_deconv': 31, # must be odd number
    'sampling': 1/5,
    'n_cores':16
}

if not exists(params['out_directory']):
    mkdir(params['out_directory'])

directory = Path(params['in_directory'])
files = directory.glob('*' + params['pickle_file_suffix'])
files = sorted([x.as_posix() for x in files])
files = [x for x in files if '-' in x]
files = [x for x in files if not exists(sub(params['in_directory'],params['out_directory'],sub(params['pickle_file_suffix'],params['out_pickle_suffix'],x)))]
print(len(files))
shuffle(files)

In [ ]:
vesselRadiusEstimator = VesselRadiiCalc(files,
                                        params['in_directory'],
                                        params['img_directory'],
                                        params['mean_directory'],
                                        params['std_directory'],
                                        params['out_directory'],
                                        params['pickle_file_suffix'],
                                        params['out_pickle_suffix'],
                                        params['img_suffix'],
                                        params['seg_suffix'],
                                        params['mean_suffix'],
                                        params['std_suffix'],
                                        params['neuron_distance_suffix'],
                                        params['second_channel'],
                                        params['neuron_channel'],
                                        params['psf'],
                                        params['spacing'],
                                        params['vessel_segment_limit'],
                                        params['max_pixel_value'],
                                        params['n_iter_deconv'],
                                        params['grid_size_psf_deconv'],
                                        params['sampling'],
                                        params['n_cores'])
vesselRadiusEstimator.process_all_files()